# ElasticsearchEmbeddingsCache

这将帮助您开始使用 Elasticsearch [键值存储](/docs/concepts/key_value_stores)。有关 `ElasticsearchEmbeddingsCache` 功能和配置的详细文档，请参阅 [API 参考](https://python.langchain.com/api_reference/elasticsearch/cache/langchain_elasticsearch.cache.ElasticsearchEmbeddingsCache.html)。

## 概述

`ElasticsearchEmbeddingsCache` 是一个 `ByteStore` 实现，它使用您的 Elasticsearch 实例来高效地存储和检索嵌入。

### 集成详情

| 类 | 包 | 本地 | JS 支持 | 包下载量 | 最新版本 |
| :--- | :--- | :---: | :---: |  :---: | :---: |
| [ElasticsearchEmbeddingsCache](https://python.langchain.com/api_reference/elasticsearch/cache/langchain_elasticsearch.cache.ElasticsearchEmbeddingsCache.html) | [langchain_elasticsearch](https://python.langchain.com/api_reference/elasticsearch/index.html) | ✅ | ❌ | ![PyPI - Downloads](https://img.shields.io/pypi/dm/langchain_elasticsearch?style=flat-square&label=%20) | ![PyPI - Version](https://img.shields.io/pypi/v/langchain_elasticsearch?style=flat-square&label=%20) |

## 设置

要创建 `ElasticsearchEmbeddingsCache` 字节存储，您需要一个 Elasticsearch 集群。您可以 [在本地设置一个](https://www.elastic.co/downloads/elasticsearch) 或创建 [Elastic 账户](https://www.elastic.co/elasticsearch)。

### 安装

LangChain 的 `ElasticsearchEmbeddingsCache` 集成位于 `__package_name__` 包中：

In [ ]:
%pip install -qU langchain_elasticsearch

## 实例化

现在我们可以实例化我们的字节存储：

In [4]:
from langchain_elasticsearch import ElasticsearchEmbeddingsCache

# Example config for a locally running Elasticsearch instance
kv_store = ElasticsearchEmbeddingsCache(
    es_url="https://localhost:9200",
    index_name="llm-chat-cache",
    metadata={"project": "my_chatgpt_project"},
    namespace="my_chatgpt_project",
    es_user="elastic",
    es_password="<GENERATED PASSWORD>",
    es_params={
        "ca_certs": "~/http_ca.crt",
    },
)

## 用法

您可以使用 `mset` 方法，像这样在键下设置数据：

In [5]:
kv_store.mset(
    [
        ["key1", b"value1"],
        ["key2", b"value2"],
    ]
)

kv_store.mget(
    [
        "key1",
        "key2",
    ]
)

[b'value1', b'value2']

您还可以使用 `mdelete` 方法删除数据：

In [6]:
kv_store.mdelete(
    [
        "key1",
        "key2",
    ]
)

kv_store.mget(
    [
        "key1",
        "key2",
    ]
)

[None, None]

## 用作嵌入缓存

与其他 `ByteStores` 一样，您可以将 `ElasticsearchEmbeddingsCache` 实例用于[文档摄取中的持久化缓存](/docs/how_to/caching_embeddings/)，以实现 RAG。

但是，默认情况下，缓存的向量将无法搜索。开发者可以通过自定义 Elasticsearch 文档的构建过程来添加索引的向量字段。

这可以通过继承和重写方法来完成：

In [ ]:
from typing import Any, Dict, List


class SearchableElasticsearchStore(ElasticsearchEmbeddingsCache):
    @property
    def mapping(self) -> Dict[str, Any]:
        mapping = super().mapping
        mapping["mappings"]["properties"]["vector"] = {
            "type": "dense_vector",
            "dims": 1536,
            "index": True,
            "similarity": "dot_product",
        }
        return mapping

    def build_document(self, llm_input: str, vector: List[float]) -> Dict[str, Any]:
        body = super().build_document(llm_input, vector)
        body["vector"] = vector
        return body

当覆盖映射和文档构建时，请仅进行附加修改，保持基础映射不变。

## API 参考

有关 `ElasticsearchEmbeddingsCache` 所有功能和配置的详细文档，请访问 API 参考：https://python.langchain.com/api_reference/elasticsearch/cache/langchain_elasticsearch.cache.ElasticsearchEmbeddingsCache.html